<a href="https://colab.research.google.com/github/dqj5182/Sparse_COO_Tensor_Multiplication_Pytorch/blob/main/SCTM_colab_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np

Sparse Matrix Multiplication Function

In [ ]:
def sparse_coo_mul(tensor1, tensor2):
  tensor1_indices = tensor1.coalesce().indices()
  tensor2_indices = tensor2.coalesce().indices()
  tensor1_values = tensor1.coalesce().values()
  tensor2_values = tensor2.coalesce().values()

  def swap_two_rows(tensor):
    index = [1, 0]
    tensor[index,] = tensor.clone()
    return tensor

  test_1 = tensor1_indices.t()
  test_2 = swap_two_rows(tensor2_indices).t()

  mul_i_j = []

  final = {}

  final_list = []

  for i, val_i in enumerate(test_1):
    for j, val_j in enumerate(test_2):

      if torch.equal(val_i[1], val_j[1]):
        mul_i_j.append([val_i[0].item(), val_j[0].item(), (tensor1_values[i]*tensor2_values[j]).item()]) # val_i[0]: row 1, val_i[1]: col 1, val_j[0]: col 2, val_j[1]: row 2

  for ele1, ele2, ele3 in mul_i_j:

    if (ele1, ele2) not in final:
      final[(ele1, ele2)] = ele3
    else:
      final[(ele1, ele2)] = final[(ele1, ele2)] + ele3

  for key, value in final.items():
    temp = [key[0], key[1] ,value]
    final_list.append(temp)

  final_list = np.array(final_list).T

  new_index = final_list[0:2,]
  new_value = final_list[2,]

  new_sparse_coo_tensor = torch.sparse_coo_tensor(new_index, new_value)
  
  return new_sparse_coo_tensor

In [ ]:
i_1 = torch.tensor([[0, 1, 1],
                  [2, 1, 2]])
v_1 = torch.tensor([3, 4, 5], dtype=torch.float32)

a = torch.sparse_coo_tensor(i_1, v_1)
a

tensor(indices=tensor([[0, 1, 1],
                       [2, 1, 2]]),
       values=tensor([3., 4., 5.]),
       size=(2, 3), nnz=3, layout=torch.sparse_coo)

In [ ]:
i_2 = torch.tensor([[2, 1, 1],
                  [1, 0, 1]])
v_2 = torch.tensor([1, 7, 2], dtype=torch.float32)

b = torch.sparse_coo_tensor(i_2, v_2)
b

tensor(indices=tensor([[2, 1, 1],
                       [1, 0, 1]]),
       values=tensor([1., 7., 2.]),
       size=(3, 2), nnz=3, layout=torch.sparse_coo)

In [ ]:
sparse_coo_mul(a, b)

tensor(indices=tensor([[0, 1, 1],
                       [1, 0, 1]]),
       values=tensor([ 3., 28., 13.]),
       size=(2, 2), nnz=3, dtype=torch.float64, layout=torch.sparse_coo)